In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer, SpatialDropout1D, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.math import confusion_matrix

## Load data

In [ ]:
def to_3D(input_data):
    print("Before reshaping: ", input_data.shape)
    X = []
    for row in input_data: 
        X.append([row])
    X = np.asarray(X)
    print("After reshaping: ", X.shape)
    return X

In [ ]:
features = np.loadtxt('data/mfcc_mel/features.txt')
targets = np.loadtxt('data/mfcc_mel/targets.txt')
features_3D = to_3D(features)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_3D, targets, test_size=0.2, random_state=1)

## Model

In [ ]:
def create_model():
    model = Sequential()
    model.add(InputLayer(input_shape=(1, 145)))
    model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences = True)))
    model.add(SpatialDropout1D(0.1))
    model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences = True)))
    model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences = True)))
    model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.1)))
    model.add(Dense(6, activation='softmax'))
    opt = Adam(learning_rate=0.00001)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)
    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
earlystopping = EarlyStopping(patience=10,restore_best_weights=True)
checkpoint_path = "ckpts/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)
batch_size = 32
epochs = 150

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
         validation_split=0.15,
         callbacks=[cp_callback])

## Metrics

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
accuracy = model.evaluate(X_test, y_test)
print('Loss: {:0.3f}\nAccuracy: {:0.3f}'.format(accuracy[0],accuracy[1]))

In [ ]:
prediction = model.predict_classes(X_test)
label = np.where(y_test)[1]

In [ ]:
confusion = confusion_matrix(labels=label, predictions=prediction, num_classes=6)

In [ ]:
cate = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad']

In [ ]:
cm = pd.DataFrame(confusion.numpy(), index = [i for i in cate] , columns = [i for i in cate])
sns.heatmap(cm, linecolor='white', cmap='Blues', linewidth=1, annot=True, fmt='')
plt.title('Confusion Matrix', size=20)
plt.xlabel('Predicted Labels', size=14)
plt.ylabel('Actual Labels', size=14)
plt.show()